In [ ]:
# ! pip install kagglehub
# ! pip3 install torch torchvision
# ! pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu129 --verbose

# ! pip3 uninstall -y torch torchvision torchaudio
# ! pip3 cache purge
# ! pip list
# ! nvidia-smi

In [42]:
import kagglehub
import shutil
import os

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.optim as optim

print(torch.__version__)

destination_path = os.path.join(os.path.expanduser("~"), "Downloads")
destination_path = 'D' + destination_path[1:] + r'\KC'

2.8.0+cu129


In [44]:
if False:
    # Download latest version
    source_path = kagglehub.dataset_download("ryanholbrook/car-or-truck")
    #destination_path = r"D:\Users\fejio\Downloads\KC"
    print("Path to dataset files:", source_path)

    os.makedirs(destination_path, exist_ok=True)
    # shutil.copytree(source_path, destination_path, dirs_exist_ok=True)
    shutil.move(source_path, destination_path)
    print("Dataset copied to:", destination_path)

In [45]:
# Image size must match target_size=(64,64)
image_size = (128, 128)

# Training data augmentations (same as ImageDataGenerator)
train_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomAffine(degrees=0, shear=20),   # shear_range=0.2
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.2)), # zoom_range
    transforms.RandomHorizontalFlip(p=0.5),         # horizontal_flip
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # rescale=1./255 equivalent
])

# Validation data (only rescaling)
test_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

destination_path_1 = destination_path + r'\1'

# Datasets (folder structure must be same as Keras: class subfolders inside train/valid)
train_dataset = datasets.ImageFolder(root=destination_path_1 + r"\train", transform=train_transform)
test_dataset =  datasets.ImageFolder(root=destination_path_1 + r"\valid", transform=test_transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)




In [46]:
# Example: iterate
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 3, 128, 128]) torch.Size([32])


In [47]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model_state = model.state_dict()


In [48]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolution layer 1
        # input_shape = [64, 64, 3] → in_channels=3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1)  # pool_size=2, strides=1
        
        # Convolution layer 2
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # pool_size=2, strides=2
        
        self.flatten = nn.Flatten()
        
        # Connection layers
        self.fc1 = nn.Linear(1, 200)  # must compute flattened size
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 1)  # output layer for binary classification

    def forward(self, x):
        # Convolution layer 1
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        # Convolution layer 2
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        # Flatten Layer
        x = self.flatten(x)
        # Initialize fc1 dynamically on first forward pass
        if self.fc1.in_features == 1:
            self.fc1 = nn.Linear(x.shape[1], 200).to(x.device)

        # Connection layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))  # sigmoid like Keras
        return x

# Instantiate model
model = CNN()
print(model)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=1, bias=True)
)


In [49]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = CNN().to(device)

# Binary Cross-Entropy loss (since final layer uses sigmoid)
criterion = nn.BCELoss()

# Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 150
early_stopping = EarlyStopping(patience=150)

for epoch in range(num_epochs):
    #print(f"{epoch} of {num_epochs}")
    # === Training ===
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    
    for images, labels in train_loader:
        # Move data to GPU
        images, labels = images.to(device), labels.to(device).float().view(-1, 1)
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        preds = (outputs > 0.5).float()
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)
    
    # === Validation ===
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float().view(-1, 1)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            preds = (outputs > 0.5).float()
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    
    avg_val_loss = val_loss / len(test_loader)
    avg_val_accuracy = 100*val_correct/val_total
    avg_accuracy = 100*train_correct/train_total
    avg_loss = train_loss/len(train_loader)
    # Print epoch results
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {avg_loss:.4f}, "
          f"Train Acc: {avg_accuracy:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f}, "
          f"Val Acc: {avg_val_accuracy:.2f}%")
    
    #avg_val_loss = val_loss / len(test_loader)
    #print(f"Epoch {epoch+1}/{num_epochs}, Val Loss = {avg_val_loss:.4f}")
    early_stopping(avg_accuracy, model)

    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

# Restore best model weights
model.load_state_dict(early_stopping.best_model_state)

Using device: cuda
Epoch [1/150] Train Loss: 0.6645, Train Acc: 59.53% | Val Loss: 0.6448, Val Acc: 61.91%
Epoch [2/150] Train Loss: 0.6325, Train Acc: 63.90% | Val Loss: 0.6202, Val Acc: 64.76%
Epoch [3/150] Train Loss: 0.6268, Train Acc: 64.20% | Val Loss: 0.6161, Val Acc: 66.52%
Epoch [4/150] Train Loss: 0.6208, Train Acc: 65.06% | Val Loss: 0.6078, Val Acc: 66.70%
Epoch [5/150] Train Loss: 0.6132, Train Acc: 65.94% | Val Loss: 0.6078, Val Acc: 66.68%
Epoch [6/150] Train Loss: 0.6114, Train Acc: 66.07% | Val Loss: 0.6135, Val Acc: 65.85%
Epoch [7/150] Train Loss: 0.6122, Train Acc: 66.02% | Val Loss: 0.6033, Val Acc: 67.16%
Epoch [8/150] Train Loss: 0.6097, Train Acc: 66.19% | Val Loss: 0.6019, Val Acc: 67.35%
Epoch [9/150] Train Loss: 0.6045, Train Acc: 66.50% | Val Loss: 0.5969, Val Acc: 68.03%
Epoch [10/150] Train Loss: 0.5977, Train Acc: 67.29% | Val Loss: 0.5997, Val Acc: 68.09%
Epoch [11/150] Train Loss: 0.5975, Train Acc: 67.56% | Val Loss: 0.5930, Val Acc: 68.09%
Epoch [12/1

<All keys matched successfully>

In [50]:
import torch
from datetime import datetime

# Get current timestamp (e.g., 2025-08-29_15-42-10)
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# Save model with timestamp in filename
torch.save(model.state_dict(), f'{destination_path}\\model_weights_{timestamp}.pth')


In [51]:
import torch
from torchvision import transforms
from PIL import Image

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # match training normalization
])

# Load image
img_path = destination_path + r"\what.jpeg"
#print(img_path)
image = Image.open(img_path).convert("RGB")
image = transform(image).unsqueeze(0)  # add batch dimension (1, 3, 64, 64)

# Send to same device as model
image = image.to(device)

# Prediction
model.eval()
with torch.no_grad():
    output = model(image)
    pred = (output > 0.5).float().item()  # sigmoid threshold 0.5

# Map prediction back to class label
# Same as training_set.class_indices in Keras
class_indices = train_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_indices.items()}

prediction = idx_to_class[int(pred)]
print("Prediction:", prediction)

Prediction: Truck


In [53]:
import os
from PIL import Image
import torch
from torchvision import transforms

# Define same transform as training
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

pred_dir = destination_path + r"\pred"
print(pred_dir)

# Same mapping as training
class_indices = train_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_indices.items()}

model.eval()
correct, total = 0, 0

# Loop over train/ and valid/ subfolders
for split in ["train", "valid"]:
    split_dir = os.path.join(pred_dir, split)
    print(f"\nChecking {split.upper()} folder...")

    # Loop over class subfolders (car, truck)
    for class_name in os.listdir(split_dir):
        class_dir = os.path.join(split_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        for fname in os.listdir(class_dir):
            if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                # Ground truth = folder name
                true_label = class_name.lower()

                # Load image
                img_path = os.path.join(class_dir, fname)
                image = Image.open(img_path).convert("RGB")
                image = transform(image).unsqueeze(0).to(device)

                # Predict
                with torch.no_grad():
                    output = model(image)
                    pred = (output > 0.5).float().item()
                    pred_label = idx_to_class[int(pred)].lower()

                # Compare
                is_correct = (pred_label == true_label)
                correct += int(is_correct)
                total += 1

                #print(f"{fname} -> Predicted: {pred_label}, True: {true_label}, Correct: {is_correct}")

print(f"\nOverall Accuracy on PRED: {correct}/{total} = {100*correct/total:.2f}%")


D:\Users\fejio\Downloads\KC\pred

Checking TRAIN folder...

Checking VALID folder...

Overall Accuracy on PRED: 8166/10168 = 80.31%
